In [1]:
from sa import *
import os

OFF_MEM_FILE = "../hex_mem_128b.mem"
SAVE_DATA_ROOT_DIR = "./sa_py_data_Q_Num"
RANDOM_MEM_GEN = False
DO_PRINT = False
GENERATE_ISA = False
decode = False

def save_data(file_name:str, data:list):
    if not os.path.exists(SAVE_DATA_ROOT_DIR):
        os.makedirs(SAVE_DATA_ROOT_DIR)
    with open(os.path.join(SAVE_DATA_ROOT_DIR, file_name), "w") as fp:
        for d in data:
            fp.write(d + "\n")

In [2]:
# Instantiation
SA = SYSTOLIC_ARRAY(gen_isa=GENERATE_ISA, USE_Q_NUMBER=True, Q=4)
OFF_MEM = BRAM(depth=256, data_num=16, nbits=8)

# Generate Random Memory Data
if RANDOM_MEM_GEN:
    gen_random_hex_mem_data(OFF_MEM_FILE, depth=256, data_num=16, nbits=8)
OFF_MEM.load_file(file_path=OFF_MEM_FILE)

In [3]:
# 1. Write data to UB
print("1. Write data to UB")
for i in range(0, 256, 1):
    #data = encode([i - j for j in range(15, -1, -1)], 16, 8)
    #SA.WRITE_DATA(i, data)
    SA.AXI_TO_UB_INST(OFF_MEM, i, i)
    SA.GENERATE_ISA("AXI_TO_UB_INST", i*16, i*16)
data = SA.UB_PRINT(dec=decode, do_print=DO_PRINT)
save_data("1_UB.txt", data)


1. Write data to UB


In [4]:
# 2. Write weight to WB
print("2. Write weight to WB")
for i in range(0, 256, 1):
    #weight = encode([- i + j for j in range(15, -1, -1)], 16, 8)
    #SA.WRITE_WEIGHT(i, weight)
    SA.AXI_TO_WB_INST(OFF_MEM, i, 256-i-1)
    SA.GENERATE_ISA("AXI_TO_WB_INST", i*16, (256-i-1)*16)
data = SA.WB_PRINT(dec=decode, do_print=DO_PRINT)
save_data("2_WB.txt", data)

2. Write weight to WB


In [5]:

# 3. IDLE

# 4. Load Data
print("4. Load Data")
for i in range(5):
    SA.UB_TO_DATA_FIFO_INST(i)
    SA.GENERATE_ISA("UB_TO_DATA_FIFO_INST", 0, i*16)
data = SA.DATA_FIFO_PRINT(dec=decode, do_print=DO_PRINT)
save_data("4_DATA_FIFO.txt", data)

4. Load Data


In [6]:
# 5. Load Weight
print("5. Load Weight")
for i in range(21):
    SA.LOAD_WEIGHT(i)
    SA.GENERATE_ISA("UB_TO_WEIGHT_FIFO_INST", 0, i*16)

print("Print MMU's weight")
data = SA.MMU_WEIGHT_PRINT(dec=decode, do_print=DO_PRINT)
save_data("5_MMU.txt", data)

print("Print WEIGHT_FIFO's weight")
data = SA.WEIGHT_FIFO_PRINT(dec=decode, do_print=DO_PRINT)
save_data("5_WEIGHT_FIFO.txt", data)

5. Load Weight
Print MMU's weight
Print WEIGHT_FIFO's weight


In [7]:
# 6. Matrix Multiplication
for i in range(16):
    SA.MAT_MUL(i, i)
    SA.GENERATE_ISA("MAT_MUL_INST", i*16, i*16)
data = SA.ACC_PRINT(dec=decode, do_print=DO_PRINT)
save_data("6_ACC.txt", data)

In [8]:
# 7. Write result at UB
for i in range(16):
    SA.WRITE_RESULT(64 + i, i)
    SA.GENERATE_ISA("ACC_TO_UB_INST", (64 + i)*16, i*16)

# 8. Write UB's results at OFF-MEM
for i in range(16):
    SA.UB_TO_AXI_INST(OFF_MEM, i, 64 + i)
    SA.GENERATE_ISA("UB_TO_AXI_INST", i*16, (64 + i)*16)

data = OFF_MEM.print(dec=decode, do_print=DO_PRINT)
save_data("8_OFF_MEM.txt", data)

In [9]:
# 9. Matrix Multiplication with accumulation
print ("9. Matrix Multiplication with accumulation")
for i in range(16):
    SA.MAT_MUL_ACC(i, 16 + i)
    SA.GENERATE_ISA("MAT_MUL_ACC_INST", i*16, (16 + i)*16)
data = SA.ACC_PRINT(dec=decode, do_print=DO_PRINT)
save_data("9_ACC.txt", data)

9. Matrix Multiplication with accumulation


In [10]:
# 10. Write result at UB
print("10. Write result at UB")
for i in range(16):
    SA.WRITE_RESULT(i + 64 + 16, i)
    SA.GENERATE_ISA("ACC_TO_UB_INST", (64 + 16 + i)*16, i*16)
data = SA.UB_PRINT(dec=decode, do_print=DO_PRINT)
save_data("10_UB.txt", data)

10. Write result at UB


In [11]:
# 11. Write UB's result at OFF-MEM
print("11. Write UB's result at OFF-MEM")
for i in range(16, 33, 1):
    SA.UB_TO_AXI_INST(OFF_MEM, i, 64 + i)
    SA.GENERATE_ISA("UB_TO_AXI_INST", i*16, (64 + i)*16)
data = OFF_MEM.print(dec=decode, do_print=DO_PRINT)
save_data("11_OFF_MEM.txt", data)
SA.ISA_FP_CLOSE()

11. Write UB's result at OFF-MEM


In [12]:
# Check data values
compare_file(os.path.join(SAVE_DATA_ROOT_DIR, "1_UB.txt"), os.path.join(SAVE_DATA_ROOT_DIR, "1_UB_TB.txt"))
compare_file(os.path.join(SAVE_DATA_ROOT_DIR, "2_WB.txt"), os.path.join(SAVE_DATA_ROOT_DIR, "2_WB_TB.txt"))
compare_file(os.path.join(SAVE_DATA_ROOT_DIR, "4_DATA_FIFO.txt"), os.path.join(SAVE_DATA_ROOT_DIR, "4_DATA_FIFO_TB.txt"))
compare_file(os.path.join(SAVE_DATA_ROOT_DIR, "5_MMU.txt"), os.path.join(SAVE_DATA_ROOT_DIR, "5_MMU_TB.txt"))
compare_file(os.path.join(SAVE_DATA_ROOT_DIR, "5_WEIGHT_FIFO.txt"), os.path.join(SAVE_DATA_ROOT_DIR, "5_WEIGHT_FIFO_TB.txt"))
compare_file(os.path.join(SAVE_DATA_ROOT_DIR, "6_ACC.txt"), os.path.join(SAVE_DATA_ROOT_DIR, "6_ACC_TB.txt"))
compare_file(os.path.join(SAVE_DATA_ROOT_DIR, "8_OFF_MEM.txt"), os.path.join(SAVE_DATA_ROOT_DIR, "8_OFF_MEM_TB.txt"))
compare_file(os.path.join(SAVE_DATA_ROOT_DIR, "9_ACC.txt"), os.path.join(SAVE_DATA_ROOT_DIR, "9_ACC_TB.txt"))
compare_file(os.path.join(SAVE_DATA_ROOT_DIR, "10_UB.txt"), os.path.join(SAVE_DATA_ROOT_DIR, "10_UB_TB.txt"))
compare_file(os.path.join(SAVE_DATA_ROOT_DIR, "11_OFF_MEM.txt"), os.path.join(SAVE_DATA_ROOT_DIR, "11_OFF_MEM_TB.txt"))

/** Comparing "./sa_py_data_Q_Num\1_UB.txt" and "./sa_py_data_Q_Num\1_UB_TB.txt" **/
[ERROR] File not found. (./sa_py_data_Q_Num\1_UB.txt,./sa_py_data_Q_Num\1_UB_TB.txt)
"./sa_py_data_Q_Num\1_UB.txt" and "./sa_py_data_Q_Num\1_UB_TB.txt" are matching.
/** Comparing "./sa_py_data_Q_Num\2_WB.txt" and "./sa_py_data_Q_Num\2_WB_TB.txt" **/
[ERROR] File not found. (./sa_py_data_Q_Num\2_WB.txt,./sa_py_data_Q_Num\2_WB_TB.txt)
"./sa_py_data_Q_Num\2_WB.txt" and "./sa_py_data_Q_Num\2_WB_TB.txt" are matching.
/** Comparing "./sa_py_data_Q_Num\4_DATA_FIFO.txt" and "./sa_py_data_Q_Num\4_DATA_FIFO_TB.txt" **/
[ERROR] File not found. (./sa_py_data_Q_Num\4_DATA_FIFO.txt,./sa_py_data_Q_Num\4_DATA_FIFO_TB.txt)
"./sa_py_data_Q_Num\4_DATA_FIFO.txt" and "./sa_py_data_Q_Num\4_DATA_FIFO_TB.txt" are matching.
/** Comparing "./sa_py_data_Q_Num\5_MMU.txt" and "./sa_py_data_Q_Num\5_MMU_TB.txt" **/
[ERROR] File not found. (./sa_py_data_Q_Num\5_MMU.txt,./sa_py_data_Q_Num\5_MMU_TB.txt)
"./sa_py_data_Q_Num\5_MMU.txt" 

True

In [13]:
compare_file(os.path.join(SAVE_DATA_ROOT_DIR, "11_OFF_MEM.txt"), os.path.join(SAVE_DATA_ROOT_DIR, "Q_VIP_OFF_MEM_TB.txt"))

/** Comparing "./sa_py_data_Q_Num\11_OFF_MEM.txt" and "./sa_py_data_Q_Num\Q_VIP_OFF_MEM_TB.txt" **/
[WARNING] Data mismatch in 1 line.
          Data1:7f80557f807f8080809c807f80807f7f
          Data2:7f802b7f8063808080ce807f80806f7f
[WARNING] Data mismatch in 2 line.
          Data1:7f80557f807f8080809c807f80807f7f
          Data2:7f802b7f8063808080ce807f80806f7f
[WARNING] Data mismatch in 3 line.
          Data1:7f80557f807f8080809c807f80807f7f
          Data2:7f802b7f8063808080ce807f80806f7f
[WARNING] Data mismatch in 4 line.
          Data1:7f80557f807f8080809c807f80807f7f
          Data2:7f802b7f8063808080ce807f80806f7f
[WARNING] Data mismatch in 5 line.
          Data1:7f80557f807f8080809c807f80807f7f
          Data2:7f802b7f8063808080ce807f80806f7f
[WARNING] Data mismatch in 6 line.
          Data1:7f80557f807f8080809c807f80807f7f
          Data2:7f802b7f8063808080ce807f80806f7f
[WARNING] Data mismatch in 7 line.
          Data1:7f80557f807f8080809c807f80807f7f
          Data2:7f

False